In [104]:
from collections import namedtuple
from math import inf, prod
from Crypto.Util.number import inverse
from typing import List
from sympy import factorint, isprime, prime, integer_nthroot, mod_inverse
from sympy.polys.polytools import invert
from sympy import symbols, Poly, div
from sympy import primerange, sqrt, simplify, expand

Point = namedtuple("Point", "x y isinf", defaults=[False])
def pt_to_str(p: Point):
    return "Point("+str(p.x)+","+str(p.y)+")" if not p.isinf else "O"
Point.__str__ = pt_to_str

EllipticCurve = namedtuple("EllipticCurve", "p a b")

def ec_frobenius(curve: EllipticCurve, P: Point):
    """Apply the Frobenius map to point P = (x, y) on the curve."""
    return Point(pow(P.x, curve.p, curve.p), pow(P.y, curve.p, curve.p), P.isinf)

def ec_neg(curve: EllipticCurve, p: Point):
    return Point(p.x, -p.y % curve.p, p.isinf)

def ec_add(curve: EllipticCurve, p1: Point, p2: Point):
    assert not (p1.isinf and p2.isinf), "cannot add O to O"
    if p1.isinf: return p2
    if p2.isinf: return p1
    if (p1.x==p2.x and p1.y==-p2.y): return Point(0,0,isinf=True)
    if p1.x!=p2.x:
        # print(p2.x, p1.x, curve.p)
        tangent = (p2.y - p1.y)*inverse(p2.x - p1.x, curve.p) # pow(p2.x - p1.x, curve.p-2, curve.p)
    else:
        tangent = (3*(p1.x)**2 + curve.a)*inverse(2*p1.y, curve.p) # pow(2*p1.y, curve.p-2, curve.p)
    p3x = tangent**2 - p1.x - p2.x
    p3y = tangent*(p1.x - p3x) - p1.y
    return Point(p3x%curve.p, p3y%curve.p)

def ec_mul(curve: EllipticCurve, p: Point, n: int):
    if n==0: return Point(p.x, p.y, True)
    p1 = p
    p2 = Point(0,0,True)
    while (n>0):
        if n%2==1: p2 = ec_add(curve, p1, p2)
        p1 = ec_add(curve, p1, p1)
        n = n//2
    return p2

division_polynomial_cache = {}
x, y = symbols('x y')
def reduce_mod_p(poly, p):
    """Reduce a polynomial modulo p, but leave y terms untouched."""
    # Get the coefficients of the polynomial
    coeffs = poly.all_coeffs()
    # Reduce only numeric coefficients modulo p
    mod_coeffs = [c % p if c.is_number else c for c in coeffs]
    # Return a new polynomial with reduced coefficients
    return Poly.from_list(mod_coeffs, gens=poly.gens)


# Define the division polynomial function with modular arithmetic
from functools import cache
def division_polynomial(E: EllipticCurve, n):
    """Calculate the n-th division polynomial modulo p for the elliptic curve y^2 = x^3 + ax + b."""
    a, b, p = E.a, E.b, E.p
    DIVPOL_BASE_CASE = [
        Poly(0, x),
        Poly(1, x),
        Poly(2 * y, x),
        Poly(3 * x**4 + 6 * a * x**2 + 12 * b * x - a**2, x),
        Poly(4 * y * (x**6 + 5 * a * x**4 + 20 * b * x**3 - 5 * a**2 * x**2 - 4 * a * b * x - 8 * b**2 - a**3), x)
    ]
    @cache
    def f(E: EllipticCurve, n):
        # print(DIVPOL_BASE_CASE[4]*DIVPOL_BASE_CASE[2]**3 - DIVPOL_BASE_CASE[1]*DIVPOL_BASE_CASE[3]**3)
        if n in division_polynomial_cache:
            return division_polynomial_cache[n]
        if n <= 4:
            result = DIVPOL_BASE_CASE[n]
        elif n % 2 == 1:  # Odd n: psi_2k+1
            m = (n - 1) // 2
            psi_m = f(E, m)
            psi_m1 = f(E, m + 1)
            psi_m2 = f(E, m + 2)
            psi_ms1 = f(E, m - 1)
            result = psi_m2 * psi_m**3 - psi_ms1 * psi_m1**3
            # result = reduce_mod_p(result, p)  # Apply mod p reduction
        else:  # Even n: psi_2k
            m = n // 2
            psi_m = f(E, m)
            psi_m1 = f(E, m + 1)
            psi_m2 = f(E, m + 2)
            psi_ms1 = f(E, m - 1)
            psi_ms2 = f(E, m - 2)
            result = (psi_m/(2 * y))*(psi_m2*psi_ms1**2 - psi_ms2*psi_m1**2)
            # result = reduce_mod_p(result, p)  # Apply mod p reduction
        division_polynomial_cache[n] = result
        return result
    return f(E, n)

In [112]:
E = EllipticCurve(101, 1, 1)
Poly(expand(simplify(division_polynomial(E, 6).subs(y**2, (x**3 + x + 1))).subs(y**2, (x**3 + x + 1)).subs(y, 2*(x**3 + x + 1))))

[12,
 0,
 300,
 2700,
 -1168,
 2976,
 -14672,
 -19760,
 -99160,
 -120224,
 -117464,
 -247224,
 -141008,
 -151200,
 -178192,
 -121648,
 -116052,
 -86368,
 -22772,
 2444]

In [93]:
E = EllipticCurve(101, 1, 1)
# simplify(division_polynomial(E, 5).subs(y**2, (x**3 + x + 1))) # correct
expand(simplify(division_polynomial(E, 6).subs(y**2, (x**3 + x + 1))).subs(y**2, (x**3 + x + 1)).subs(y, 2*(x**3 + x + 1))) # correct
# simplify(division_polynomial(E, 7).subs(y**2, (x**3 + x + 1))) # correct
# expand(simplify(simplify(division_polynomial(E, 8).subs(y**2, (x**3 + x + 1))).subs(y**2, (x**3 + x + 1))).subs(y, 2*(x**3 + x + 1))) # correct
# simplify(division_polynomial(E, 9).subs(y**2, (x**3 + x + 1)))
# expand(simplify(simplify(division_polynomial(E, 10).subs(y**2, (x**3 + x + 1))).subs(y**2, (x**3 + x + 1))).subs(y, 2*(x**3 + x + 1)))

12*x**19 + 300*x**17 + 2700*x**16 - 1168*x**15 + 2976*x**14 - 14672*x**13 - 19760*x**12 - 99160*x**11 - 120224*x**10 - 117464*x**9 - 247224*x**8 - 141008*x**7 - 151200*x**6 - 178192*x**5 - 121648*x**4 - 116052*x**3 - 86368*x**2 - 22772*x + 2444